In [ ]:
from LODEGP.LODEGP import LODEGP, list_standard_models
import gpytorch
import torch
import matplotlib.pyplot as plt
from finite_difference import central_difference
from helpers.training_functions import granso_optimization
import numpy as np

In [ ]:
V = matrix([[                                0,                          -400/981 ,                    x^2 + 981/200]
,[                                0                  ,        -200/981             ,    1/2*x^2 + 981/200]
,[                               -1                  ,-400/981*x^2 - 4 ,x^4 + 2943/200*x^2 + 962361/20000]])

In [ ]:
# Base ODE:     
# [x**2 + 981/(100*l1), 0, -1/l1, 0, x**2+981/(100*l2), -1/l2]

In [ ]:
var("t")
base_func = 1/(40*t)*sin(t)
p = matrix([[0],[0],[base_func]])
f = V*p
print(f)

f0 = 1/200*(200*base_func.diff(t, 2) + 981*base_func)
f1 = 1/200*(100*base_func.diff(t, 2) + 981*base_func)
f2 = 1/20000*(20000*base_func.diff(t, 4) + 294300*base_func.diff(t, 2) + 981**2*base_func)
plot(f1, (t, -10, 10), color='red')
f1

In [ ]:
# Check if both ODEs are satisfied
print((f0.diff(t, 2) + 9.81*f0 - f2))
print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))

In [ ]:
START = 2r
END = 12r
COUNT = 100r
train_x = torch.linspace(START, END, COUNT)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=5r)

y0_func = lambda x: float(781/8000)*torch.sin(x)/x - float(1/20)*torch.cos(x)/x**2 + float(1/20)*torch.sin(x)/x**3
y1_func = lambda x: float(881/8000)*torch.sin(x)/x - float(1/40)*torch.cos(x)/x**2 + float(1/40)*torch.sin(x)/x**3
y2_func = lambda x: float(688061/800000)*torch.sin(x)/x - float(2543/4000)*torch.cos(x)/x**2 + float(1743/4000)*torch.sin(x)/x**3 - float(3/5)*torch.cos(x)/x**4 + float(3/5)*torch.sin(x)/x**5 
y0 = y0_func(train_x)
y1 = y1_func(train_x)
y2 = y2_func(train_x)
train_y = torch.stack([y0, y1, y2], dim=-1r)

model = LODEGP(train_x, train_y, likelihood, 3, ODE_name="Bipendulum", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})
model = LODEGP(train_x, train_y, likelihood, 5, ODE_name="Three tank", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})

In [ ]:
values = list()
differences = list()
for value in train_x:
    cd_val = float(central_difference(y0_func, value, h=1e-1r, precision=6, order=1))
    compalg_val = f0.diff(t, 1)(t=float(value)).n()
    values.append((cd_val, compalg_val))
    differences.append(abs(cd_val - compalg_val))
differences

In [ ]:
# ODE 1
#(f0.diff(t, 2) + 9.81*f0 - f2)
h = 1e-1r
print("ODE1 median error: \t" , np.median(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))
print("ODE1 avg. error: \t" , np.average(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))



# ODE 2
#(f1.diff(t, 2) + 9.81/2*f1 - f2/2)

print("ODE2 median error: \t" , np.median(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))
print("ODE2 avg. error: \t" , np.average(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))

In [ ]:
# Plot the posterior GP and the data
model.eval()
model.likelihood.eval()
with torch.no_grad():
    test_x = torch.linspace(2r, 12r, 100r)
    observed_pred = model.likelihood(model(test_x))
    observed_pred_mean = observed_pred.mean
    observed_pred_var = observed_pred.covariance_matrix.diag().reshape(-1, 3)
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for i in range( 3):
        ax.plot(test_x.numpy(), observed_pred_mean[:, i].numpy(), 'k')
        ax.fill_between(test_x.numpy(),
                        observed_pred_mean[:, i].numpy() - 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        observed_pred_mean[:, i].numpy() + 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        alpha=0.5)
        ax.scatter(train_x.numpy(), train_y[:, i].numpy(), s=10, c='r', marker='x')
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    ax.set_title('Posterior GP')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()

In [ ]:
from helpers.util_functions import get_full_kernels_in_kernel_expression

#for param in model.state_dict():
#    print(param)
#    print(model.state_dict()[param])
#list(model.named_hyperparameters())
list(model.covar_module.named_children())
k = gpytorch.kernels.RBFKernel() + gpytorch.kernels.RBFKernel()
for name, sub_kernel in k.named_children():
    print(name)
    print(sub_kernel)

In [ ]:
import torch
import numpy as np


# standard gpytorch class
class ExactGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())


class ExactGP2(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP2, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel())


class ExactGP3(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP3, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel()) + gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())



def sample_value(shape, bounds, dist_type):
    """Sample values from a given distribution type."""
    if dist_type == "log-uniform":
        log_bounds = np.log(bounds)
        return torch.tensor(
            np.exp(np.random.uniform(*log_bounds, size=shape)),
            dtype=torch.float32
        )
    elif dist_type == "uniform":
        return torch.tensor(
            np.random.uniform(*bounds, size=shape),
            dtype=torch.float32
        )
    else:
        raise ValueError(f"Unsupported distribution type: {dist_type}")

def randomize_model_hyperparameters(
    model,
    param_specs,
    default_bounds=(0.1, 1.0),
    default_type="uniform",
    verbose=False
):
    """
    Randomly reinitialize model hyperparameters using specified or fallback distributions.

    Parameters:
    - model: GP model with .named_hyperparameters()
    - param_specs: dict mapping param names to:
          {
              "bounds": (lower, upper),
              "type": optional, overrides default_type
          }
    - default_bounds: fallback bounds if param not in param_specs
    - default_type: fallback sampling distribution ("uniform" or "log-uniform")
    - verbose: whether to print changes
    """

    for name, param in model.named_hyperparameters():
        spec = param_specs.get(name, {})

        bounds = spec.get("bounds", default_bounds)
        dist_type = spec.get("type", default_type)

        shape = param.shape
        new_value = sample_value(shape, bounds, dist_type)

        with torch.no_grad():
            param.copy_(new_value)

        if verbose:
            print(f"[Reinit] {name} ← {new_value.cpu().numpy()} (dist: {dist_type}, bounds: {bounds})")



param_specs = {
    "likelihood.noise": {"bounds": (1e-4, 1e-1)},
    "covar_module.outputscale": {"bounds": (0.5, 2.0)}
}

param_specs = {
    "likelihood.noise": {"bounds": (1e-4, 1e-1)},  # log-uniform by default
    "covar_module.outputscale": {
        "bounds": (0.5, 2.0),
        "type": "uniform"  # override to linear
    },
    "model_parameters.signal_variance_2": {
        "bounds": (0.5, 2.0),
        "type": "uniform"  # override to linear
    }
}



RBF_GP = ExactGP(train_x, train_y, likelihood)
Mat_GP = ExactGP2(train_x, train_y, likelihood)
Sum_GP = ExactGP3(train_x, train_y, likelihood)


randomize_model_hyperparameters(
    Mat_GP,
    param_specs,
    default_bounds=(0.1, 5.0),
    default_type="log-uniform",
    verbose=True
)
print("====")
randomize_model_hyperparameters(
    RBF_GP,
    param_specs,
    default_bounds=(0.1, 5.0),
    default_type="log-uniform",
    verbose=True
)


In [ ]:
def sample_value(shape, bounds, dist_type):
    """Sample values from a given distribution type."""
    if dist_type == "log-uniform":
        log_bounds = np.log(bounds)
        return torch.tensor(
            np.exp(np.random.uniform(*log_bounds, size=shape)), dtype=torch.float32
        )
    elif dist_type == "uniform":
        return torch.tensor(
            np.random.uniform(*bounds, size=shape), dtype=torch.float32
        )
    else:
        raise ValueError(f"Unsupported distribution type: {dist_type}")



def randomize_model_hyperparameters(
    model,
    param_specs=None,
    kernel_param_specs=None,
    default_bounds=(0.1, 1.0),
    default_type="uniform",
    verbose=False
):
    """
    Reinitializes model hyperparameters using a mix of exact param specs and kernel-based specs.

    Parameters:
    - model: GP model with .named_hyperparameters()
    - param_specs: dict keyed by full param name
    - kernel_param_specs: dict keyed by (kernel_type, param_name)
    - default_bounds: fallback bounds
    - default_type: "uniform" or "log-uniform"
    - verbose: if True, prints sampled values
    """
    param_specs = param_specs or {}
    kernel_param_specs = kernel_param_specs or {}

    # Step 1: Get kernel types in order
    kernel_types = get_full_kernels_in_kernel_expression(model.covar_module)
    print(f"Kernel types: {kernel_types}")
    kernel_index = 0  # to match hyperparameters to kernel types sequentially

    for name, param in model.named_hyperparameters():
        print(f"Processing {name}...")
        shape = param.shape

        # Case 1: Full-name match
        if name in param_specs:
            spec = param_specs[name]
            bounds = spec.get("bounds", default_bounds)
            dist_type = spec.get("type", default_type)
            kernel_type = "<unknown>"

        # Case 2: Try kernel-type-based matching
        else:
            # Extract the local param name (e.g., "lengthscale" from "covar_module.base_kernel.lengthscale")
            local_param_name = name.split(".")[-1]
            if kernel_index < len(kernel_types):
                kernel_type = kernel_types[kernel_index]
                kernel_index += 1
                spec = kernel_param_specs.get((kernel_type, local_param_name), {})
                bounds = spec.get("bounds", default_bounds)
                dist_type = spec.get("type", default_type)
            else:
                kernel_type = "<unknown>"
                bounds = default_bounds
                dist_type = default_type

        # Sample and assign
        new_value = sample_value(shape, bounds, dist_type)
        with torch.no_grad():
            param.copy_(new_value)

        if verbose:
            print(f"[Reinit] {name} ← {new_value.cpu().numpy()} (kernel: {kernel_type}, dist: {dist_type}, bounds: {bounds})")


kernel_param_specs = {
    ("RBFKernel", "raw_lengthscale"): {"bounds": (0.1, 2.0), "type": "log-uniform"},
    ("MaternKernel", "raw_lengthscale"): {"bounds": (1.0, 10.0), "type": "log-uniform"},
    ("ScaleKernel", "raw_outputscale"): {"bounds": (0.5, 2.0), "type": "uniform"}
}

param_specs = {
    "likelihood.raw_task_noises": {"bounds": (1e-4, 1e-1)},
    "likelihood.raw_noise": {"bounds": (1e-4, 1e-1)}
}

randomize_model_hyperparameters(
    model=model,
    param_specs=param_specs,
    kernel_param_specs=kernel_param_specs,
    default_bounds=(0.1, 1.0),
    default_type="log-uniform",
    verbose=True
)


In [ ]:
# standard gpytorch class
class ExactGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())


class ExactGP2(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP2, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel())


class ExactGP3(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGP3, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel()) + gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())


RBF_GP = ExactGP(train_x, train_y, likelihood)
Mat_GP = ExactGP2(train_x, train_y, likelihood)
Sum_GP = ExactGP3(train_x, train_y, likelihood)

print(get_full_kernels_in_kernel_expression(RBF_GP.covar_module))
# > ['ScaleKernel', 'RBFKernel']
print(get_full_kernels_in_kernel_expression(Mat_GP.covar_module))
# > ['ScaleKernel', 'MaternKernel']
print(get_full_kernels_in_kernel_expression(Sum_GP.covar_module))
# > ['ScaleKernel', 'MaternKernel', 'ScaleKernel', 'RBFKernel']
print(get_full_kernels_in_kernel_expression(model.covar_module))
# > ['LODE_Kernel']